In [1]:
import numpy as np
import pandas as pd
import re
import os
import glob
import warnings

from analysis_functions import *

import plotly.graph_objects as go
import plotly.colors as colors
import plotly.io as pio

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
# Define the pattern for the files
trial_pattern = re.compile(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}_trials\.csv')
track_pattern = re.compile(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}_eyetracking\.csv')


# Use glob to get all files in the folder
files = glob.glob('C:/Users/hssla/Downloads/avery_test/feedback/*')

# Filter files using the regular expression
trial_files = [f for f in files if trial_pattern.match(os.path.basename(f))]
track_files = [f for f in files if track_pattern.match(os.path.basename(f))]

# Function to read and concatenate CSV files with error handling
def read_and_concatenate(files, sep=','):
    dfs = []
    for ii, file in enumerate(files):
        try:
            df = pd.read_csv(file, on_bad_lines='skip', sep=sep)
            df['Session'] = ii
            dfs.append(df)
        except pd.errors.ParserError as e:
            print(f"Error parsing {file}: {e}")
    return pd.concat(dfs, ignore_index=True), ii+1  # Return dataframe and session count

# Load and concatenate the trial files
trial_data, session_count = read_and_concatenate(trial_files)

# Load and concatenate the track files
track_data, session_count = read_and_concatenate(track_files, sep=';')


In [5]:
trial_data.head()

,Block,Trial,Reaction_Time,Response,Coherence,Direction,Key_Pressed,Radius,Theta,X,Y,Seed,Speed,Size,Lifespan,FadeLength,Session
0,0,0,6.053226,True,0.50,left,LeftArrow,0,0,0,0,8627,1,10,60,30,0
1,0,1,2.188286,True,0.49,down,DownArrow,0,0,0,0,6512,1,10,60,30,0
2,0,2,1.409954,True,0.49,down,DownArrow,0,0,0,0,2979,1,10,60,30,0
3,0,3,1.276924,True,0.48,up,UpArrow,0,0,0,0,10737,1,10,60,30,0
4,0,4,1.655300,True,0.48,right,RightArrow,0,0,0,0,5895,1,10,60,30,0


In [6]:
track_data.head()

,Frame,CaptureTime,LogTime,HMDPosition,HMDRotation,GazeStatus,CombinedGazeForward,CombinedGazePosition,InterPupillaryDistanceInMM,LeftEyeStatus,...,RightEyeForward,RightEyePosition,RightPupilIrisDiameterRatio,RightPupilDiameterInMM,RightIrisDiameterInMM,FocusDistance,FocusStability,TrialNumber,FixationPosition,Session
0,32659779,1000163117304204700,63865799939221,"(-0.230, 1.067, 0.205)","(0.315, 0.035, 0.038, 0.948)",INVALID,NaN,NaN,NaN,INVALID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(5.00, 0.00, -15.00)",0
1,32659799,1000163117404243100,63865799939223,"(-0.230, 1.067, 0.205)","(0.315, 0.035, 0.038, 0.948)",INVALID,NaN,NaN,NaN,INVALID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(5.00, 0.00, -15.00)",0
2,32659819,1000163117504202600,63865799939223,"(-0.230, 1.067, 0.205)","(0.315, 0.035, 0.038, 0.948)",INVALID,NaN,NaN,NaN,INVALID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(5.00, 0.00, -15.00)",0
3,32659839,1000163117604215100,63865799939223,"(-0.230, 1.067, 0.205)","(0.315, 0.035, 0.038, 0.948)",INVALID,NaN,NaN,NaN,INVALID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(5.00, 0.00, -15.00)",0
4,32659859,1000163117704202700,63865799939223,"(-0.230, 1.067, 0.205)","(0.315, 0.035, 0.038, 0.948)",INVALID,NaN,NaN,NaN,INVALID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(5.00, 0.00, -15.00)",0


In [7]:
# Clean up NaN trials at end of file
track_data = track_data.dropna(subset=['TrialNumber'])

In [8]:
def create_histogram(dataframe, nbins):
    # Convert boolean 'Response' column to float
    dataframe['Response'] = dataframe['Response'].astype(float)
    
    # Create bins for 'Coherence'
    dataframe['Coherence_bin'] = pd.cut(dataframe['Coherence'], bins=nbins)
    
    # Group by the new bins and calculate the mean of 'Response' and the count of observations
    grouped_df = dataframe.groupby('Coherence_bin').agg(
        average_response=('Response', 'mean'),
        count=('Response', 'size')).reset_index()
    
    # Convert bins to string for better labeling
    grouped_df['Coherence_bin'] = grouped_df['Coherence_bin'].astype(str)
    
    # Create the bar plot using Plotly
    fig = px.bar(grouped_df, x='Coherence_bin', y='average_response', 
                 title='Histogram of Coherence vs Average Response',
                 labels={'Coherence_bin': 'Coherence', 'average_response': 'Average Response Accuracy'})
    
    # Add count as text annotation
    fig.update_traces(text=grouped_df['count'], textposition='outside')
    
    # Show the plot
    fig.show()

In [7]:
create_histogram(trial_data, nbins=12)

C:\Users\hssla\AppData\Local\Temp\ipykernel_51696\2911229733.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df = dataframe.groupby('Coherence_bin').agg(


In [9]:
def visualize_logistic_regression(dataframe):
    # Convert boolean 'Response' column to float
    dataframe['Response'] = dataframe['Response'].astype(float)
    
    # Prepare the data
    X = dataframe[['Coherence']].values
    y = dataframe['Response'].values
    
    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)
    
    # Fit the logistic regression model
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Generate values for the logistic regression curve
    X_values = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
    X_values_scaled = scaler.transform(X_values)
    y_values = model.predict_proba(X_values_scaled)[:, 1]
    
    # Create the scatter plot of the original data points
    scatter = go.Scatter(x=dataframe['Coherence'], y=dataframe['Response'], mode='markers', name='Data points')
    
    # Create the logistic regression curve
    curve = go.Scatter(x=X_values.flatten(), y=y_values, mode='lines', name='Logistic Regression Curve')
    
    # Combine the scatter plot and the logistic regression curve
    fig = go.Figure(data=[scatter, curve])
    
    # Update the layout of the plot
    fig.update_layout(title='Logistic Regression Visualization',
                      xaxis_title='Coherence',
                      yaxis_title='Response',
                      showlegend=True)
    
    # Show the plot
    fig.show()

In [9]:
visualize_logistic_regression(trial_data)

### Trial Eye Movement

In [10]:
# Get correct trials in which coherence was near threshold
trial_data_focused = trial_data.loc[(trial_data['Response'] == 1.0) & (trial_data['Coherence'] < 0.33)]

In [11]:
# Get list of correct trials to filter tracking data
trials_focused = list(trial_data_focused['Trial'].astype(float))
print(*trials_focused)

39.0 40.0 41.0 42.0 43.0 44.0 45.0 47.0 48.0 49.0 94.0 95.0 96.0 97.0 98.0 99.0 100.0 101.0 102.0 139.0 140.0 143.0 144.0 145.0 146.0 147.0 148.0 149.0 151.0 152.0 153.0 154.0 155.0


In [12]:
track_data_focused = track_data.loc[track_data['TrialNumber'].isin(trials_focused)]

In [13]:
track_data_focused.head(3)

,Frame,CaptureTime,LogTime,HMDPosition,HMDRotation,GazeStatus,CombinedGazeForward,CombinedGazePosition,InterPupillaryDistanceInMM,LeftEyeStatus,...,RightEyeForward,RightEyePosition,RightPupilIrisDiameterRatio,RightPupilDiameterInMM,RightIrisDiameterInMM,FocusDistance,FocusStability,TrialNumber,FixationPosition,Session


In [14]:
gaze_data = pd.DataFrame()
gaze_data[['x', 'y']] = track_data_focused['CombinedGazeForward'].str.extract(r'\(([^,]+), ([^,]+), [^)]+\)').astype(float)
gaze_data[['TrialNumber', 'HMDRotation', 'Frame']] = track_data_focused[['TrialNumber', 'HMDRotation', 'Frame']]
gaze_data
gaze_data

,x,y,TrialNumber,HMDRotation,Frame
20212,0.035,-0.040,35.0,"(0.057, -0.054, 0.000, 0.997)",138044
20213,0.035,-0.040,35.0,"(0.057, -0.054, 0.000, 0.997)",138045
20214,0.035,-0.040,35.0,"(0.057, -0.054, 0.001, 0.997)",138046
20215,0.035,-0.039,35.0,"(0.057, -0.054, 0.000, 0.997)",138047
20216,0.035,-0.039,35.0,"(0.057, -0.054, 0.000, 0.997)",138048
...,...,...,...,...,...
218283,0.058,0.028,263.0,"(0.046, -0.064, 0.001, 0.997)",336116
218284,0.058,0.029,263.0,"(0.046, -0.064, 0.001, 0.997)",336117
218285,0.058,0.030,263.0,"(0.046, -0.064, 0.001, 0.997)",336118
218286,0.058,0.031,263.0,"(0.047, -0.064, 0.001, 0.997)",336119


In [15]:
# Save DataFrame to CSV
gaze_data.to_csv('gaze_data.csv', index=False)

In [16]:
def visualize_trial_gaze(trial_number, df, color_code="Frame"):
    trial_number = float(trial_number)
    df = df.loc[df['TrialNumber'] == trial_number].copy()
    df['Frame'] = df['Frame'] - df['Frame'].min() # Start counting frames from 0

    # Create the scatter plot with Plotly
    fig = px.scatter(df, x='x', y='y', color=color_code, title=f"2D Scatter Plot of Gaze Coordinates<br><b>Trial {int(trial_number)}</b>")

    # Center the plot at (0, 0)
    fig.update_layout(
        xaxis=dict(range=[-0.5, 0.5], zeroline=True, zerolinewidth=2, zerolinecolor='LightPink'),
        yaxis=dict(range=[-0.5, 0.5], zeroline=True, zerolinewidth=2, zerolinecolor='LightPink'),
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        yaxis_scaleanchor='x',
        width=800,
        height=800
    )

    # Show the plot
    fig.show()

In [17]:
visualize_trial_gaze(trials_focused[10], gaze_data, color_code="Frame")

In [18]:
import plotly.graph_objects as go

def visualize_trial_gaze_distance(trial_number, df, color_code="Frame", color_scale='viridis'):
    trial_number = float(trial_number)
    df = df.loc[df['TrialNumber'] == trial_number].copy()

    # Calculate the differences between consecutive x and y coordinates
    df['dx'] = df['x'].diff()
    df['dy'] = df['y'].diff()

    # Compute the Euclidean distance for each frame transition
    df['Distance'] = np.sqrt(df['dx']**2 + df['dy']**2)

    # Sum up all the distances to get the total distance traveled
    total_distance = df['Distance'].sum()

    # Create the scatter plot with Plotly
    fig = px.scatter(df, x='x', y='y', opacity=0.5, color=df[color_code], color_continuous_scale=color_scale, title=f"2D Scatter Plot of Gaze Coordinates<br><b>Trial {int(trial_number)}</b><br>Total distance: {total_distance:.2f}")
    #fig = px.line(df, x='x', y='y', color=df[color_code], title=f"2D Scatter Plot of Gaze Coordinates<br><b>Trial {int(trial_number)}</b><br>Total distance: {total_distance:.2f}")

    df = df.sort_values(by="x")
    # Center the plot at (0, 0)
    fig.update_layout(
        xaxis=dict(range=[-0.5, 0.5], zeroline=True, zerolinewidth=2, zerolinecolor='LightPink'),
        yaxis=dict(range=[-0.5, 0.5], zeroline=True, zerolinewidth=2, zerolinecolor='LightPink'),
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        yaxis_scaleanchor='x',
        width=1024,
        height=1024
    )

    # Show the plot
    fig.show()

In [19]:
fig = visualize_trial_gaze_distance(trials_focused[10], gaze_data, color_code='Distance')

### Isolate Eye Movement States

In [20]:
trial_46 = gaze_data.loc[gaze_data['TrialNumber'] == trials_focused[10]].copy()

# Calculate the differences between consecutive x and y coordinates
trial_46['dx'] = trial_46['x'].diff()
trial_46['dy'] = trial_46['y'].diff()
trial_46['Distance'] = np.sqrt(trial_46['dx']**2 + trial_46['dy']**2)

trial_46 = trial_46[1:]

In [21]:
from scipy.ndimage import gaussian_filter

trial_46['smoothed_dist'] = gaussian_filter(trial_46['Distance'], sigma=1)

In [22]:
smooth = gaussian_filter(trial_46['Distance'], sigma=2)

In [23]:
smooth

array([0.0006108 , 0.00060454, 0.00061447, ..., 0.00032667, 0.00033165,
       0.00033285])

In [24]:
# Eye movement conditions
eye_movement_conditions = [
    (trial_46['Distance'] > .01),
    (trial_46['Distance'] <= .01) & (trial_46['Distance'] > 0),
    (trial_46['Distance'] == 0)
]

eye_movement_classes = [1,2,3]

trial_46['eye_movement'] = np.select(eye_movement_conditions, eye_movement_classes, default=0)


# Eye movement conditions
eye_movement_conditions_smoothed = [
    (trial_46['smoothed_dist'] > .01),
    (trial_46['smoothed_dist'] <= .01) & (trial_46['smoothed_dist'] > 0),
    (trial_46['smoothed_dist'] == 0)
]

eye_movement_classes = [1,2,3]

trial_46['em_smoothed'] = np.select(eye_movement_conditions_smoothed, eye_movement_classes, default=0)

In [25]:
trial_46.head()

,x,y,TrialNumber,HMDRotation,Frame,dx,dy,Distance,smoothed_dist,eye_movement,em_smoothed
36318,-0.017,-0.040,46.0,"(0.079, -0.053, 0.003, 0.995)",154150,0.0,-0.001,0.001,0.000699,2,2
36319,-0.017,-0.040,46.0,"(0.079, -0.053, 0.003, 0.996)",154151,0.0,0.000,0.000,0.000543,3,2
36320,-0.017,-0.041,46.0,"(0.079, -0.053, 0.003, 0.996)",154152,0.0,-0.001,0.001,0.000516,2,2
36321,-0.017,-0.041,46.0,"(0.079, -0.053, 0.003, 0.996)",154153,0.0,0.000,0.000,0.000547,3,2
36322,-0.017,-0.042,46.0,"(0.079, -0.053, 0.003, 0.996)",154154,0.0,-0.001,0.001,0.000749,2,2


In [26]:
max(trial_46['em_smoothed'])

3

In [27]:
em_state = list(trial_46['eye_movement'])
em_state_sm = list(trial_46['em_smoothed'])

In [28]:
states = [1, 2, 3]
counts = [0, 0, 0]
state_seq = [0 for ii in range(len(em_state))]

for ii in range(1, len(em_state)):
    for kk in range(len(states)):
        if em_state[ii] == states[kk]:
            if em_state[ii] != em_state[ii-1]:
                counts[kk] += 1
            state_seq[ii] = counts[kk]

In [29]:
states = [1, 2, 3]
counts = [0, 0, 0]
state_seq_sm = [0 for ii in range(len(em_state_sm))]

for ii in range(1, len(em_state_sm)):
    for kk in range(len(states)):
        if em_state_sm[ii] == states[kk]:
            if em_state_sm[ii] != em_state_sm[ii-1]:
                counts[kk] += 1
            state_seq_sm[ii] = counts[kk]

In [31]:
#print(*em_state_seq)
print(*state_seq)
print(*state_seq_sm)
#print(state_seq == em_state_seq)

0 1 1 2 2 2 2 3 3 3 3 3 1 1 4 4 4 4 4 4 4 4 4 4 4 4 5 5 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 10 10 10 10 10 10 11 11 12 12 13 13 13 14 14 14 15 15 15 15 16 16 16 16 16 16 17 17 17 17 17 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 18 18 18 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19 20 20 20 20 20 20 21 21 22 22 22 22 22 23 23 23 23 23 23 23 23 23 24 24 24 24 24 24 24 25 25 25 25 25 25 26 26 26 26 26 26 27 27 27 27 27 28 28 28 28 29 29 30 30 30 30 30 31 31 31 31 31 31 31 31 32 32 32 32 33 33 34 34 34 34 34 35 35 35 35 35 35 35 36 36 36 36 36 36 37 37 37 37 37 37 38 38 38 38 38 38 38 39 39 39 39 39 39 39 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 41 41 41 41 41 41 42 42 42 42 42 42 42 42 42 43 43 43 43 43 44 44 44 45 45 45 46 46 46 46 47 47 48 48 48 49 49 49 49 50 50 50 50 51 51 51 51 51 51 52 52 53 53 53 53 53 53 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 55 55 5

In [32]:
trial_46['em_state_seq'] = state_seq
trial_46['em_state_seq_sm'] = state_seq_sm

In [38]:
fig = visualize_trial_gaze_distance(46, trial_46, color_code='eye_movement', color_scale='purpor')

In [35]:
fig = visualize_trial_gaze_distance(46, trial_46, color_code='em_state_seq_sm', color_scale='rainbow')

#### Single Smooth Pursuit

In [54]:
trial_46[0:1]

,x,y,TrialNumber,HMDRotation,Frame,dx,dy,Distance,smoothed_dist,eye_movement,em_smoothed,em_state_seq,em_state_seq_sm
36318,-0.017,-0.04,46.0,"(0.079, -0.053, 0.003, 0.995)",154150,0.0,-0.001,0.001,0.000699,2,2,0,0


In [64]:
smooth_only = trial_46.loc[(trial_46['em_state_seq_sm'] == 83) & (trial_46['em_smoothed'] == 2)]

# final_pursuit = trial_46.iloc[trial_46['em_state_seq']]

fig = visualize_trial_gaze_distance(46, smooth_only, color_code='em_state_seq_sm', color_scale='rainbow')